<a href="https://colab.research.google.com/github/ezhilnikhilan/CS583_QuantumAlgorithms/blob/main/Deutsch_Jozsa_n2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq

In [255]:
import cirq
import random
from cirq import H, X, CNOT, measure, I

# **Deutsch-Jozsa Algorithm for n = 2** 

#### **Oracle Logic:** The following is the most concise representation of the gates that produce $ t ⊕ f(c) $ that I could come up with:


> **FIRST 2 BITS:**
0 -> none; 
1 -> CNOT(q0), CNOT(q1);
 2 -> CNOT(q0), CNOT(q1), X; 
 3 -> CNOT(q0), X 

> **NEXT 2 BITS:** if bot > 0 && (top == 3 or bot == top) : CNOT(q0)

(Interpretations/observations after code)

In [260]:
def make_oracle_n2(q0, q1, target, secret_function):
    """Gates implementing the secret function f(x)."""
    gates = []
    top, bot = secret_function[0], secret_function[1]

    if top > 0:
        gates.append(CNOT(q0, target))
        if top > 1:
          gates.append(X(target))
        if top < 3:
          gates.append(CNOT(q1, target))
      
    if bot > 0 and (bot == top or top == 3):
       gates.append(CNOT(q0, target))
    
    yield gates

def make_deutsch_jozsa_circuit(q0, q1, target, oracle):
    c = cirq.Circuit()

    # Initialize qubits.
    c.append([X(target), H(target), H(q1), H(q0)])

    # Query oracle.
    c.append(oracle)

    # Measure in H basis.-
    c.append([H(q0), H(q1), measure([q0,q1], key = "result_q0_q1")])
    return c

def num_to_bin(num):
    digs = [dig for dig in str(bin(num)[2:])]
    if len(digs) < 2:
      digs = ["0"] + digs 
    return ", ".join(digs)


def execute_deutsch_jozsa(only_balanced = False, secret_function = [1,0], num_reps = 1):
    # Choose qubits to use.
    q0, q1, target = cirq.LineQubit.range(3)

    # Pick a secret 4-bit function and create a circuit to query the oracle. Select only the valid cases.
    while((bin(secret_function[0]).count('1') + bin(secret_function[1]).count('1')) % 2 != 0 or 
      (only_balanced and secret_function[0]%3 == secret_function[1]%3 and secret_function[0]%3 == 0)):
      secret_function = [random.randint(0, 3) for _ in range(2)]

    oracle = make_oracle_n2(q0, q1, target, secret_function)
    print(f"Secret function:\tf(x) = <{', '.join([num_to_bin(e) for e in secret_function])}>  ----> \n")

    # Embed the oracle into a quantum circuit querying it exactly once.
    circuit = make_deutsch_jozsa_circuit(q0, q1, target, oracle)
    print(f'Circuit: \n')
    print(circuit)

    # Simulate the circuit.
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=num_reps)
    measured_res = result.measurements['result_q0_q1'][:, 0:]
    print(measured_res[0][0])
    print(f'\nResult of f(0)⊕f(1)⊕f(2) for {num_reps} repetitions : {", ".join(str(res[0])+str(res[1]) for res in measured_res)} \n')
    print("-"*100)


# Constant vs Balanced:

We should measure q0q1 -> 00 for both the constant states. 

In [261]:
# BOTH CONSTANT CASES
execute_deutsch_jozsa(secret_function=[0,0], num_reps=10)
execute_deutsch_jozsa(secret_function=[3,3], num_reps=10)

#ANY RANDOM BALANCED CASE
execute_deutsch_jozsa(only_balanced=True,num_reps=10)

Secret function:	f(x) = <0, 0, 0, 0>  ----> 

Circuit: 

0: ───H───H───M('result_q0_q1')───
              │
1: ───H───H───M───────────────────

2: ───X───H───────────────────────
0

Result of f(0)⊕f(1)⊕f(2) for 10 repetitions : 00, 00, 00, 00, 00, 00, 00, 00, 00, 00 

----------------------------------------------------------------------------------------------------
Secret function:	f(x) = <1, 1, 1, 1>  ----> 

Circuit: 

0: ───H───────@───────@───H───M('result_q0_q1')───
              │       │       │
1: ───H───H───┼───────┼───────M───────────────────
              │       │
2: ───X───H───X───X───X───────────────────────────
0

Result of f(0)⊕f(1)⊕f(2) for 10 repetitions : 00, 00, 00, 00, 00, 00, 00, 00, 00, 00 

----------------------------------------------------------------------------------------------------
Secret function:	f(x) = <1, 0, 1, 0>  ----> 

Circuit: 

                          ┌──┐
0: ───H───────@────────────@─────H───M('result_q0_q1')───
              │            

# **Some observations**

## **1. Interpretation of results**

Due to phase kickback, the operations performed on the target qubit are "kicked back" to the control qubits q0 and q1.

Since the Walsh-Hadamard transform $H^{⊗n} |x⟩$ is applied to the control qubits, we can derive that the final measurement $ |u⟩ = |00⟩ $ is only achieved for the constant case.

## **2. Possibilities and speed**

For the $ n = 2 $ case, there are a total of 8 (2 constant + 6 balanced) possible cases for the unknown function $f(x)$. A classical computer would need to make $ 2^{n-1} + 1 = 3 $ queries to $f(x)$ in order to reliably determine if it is constant or balanced.

The Deutsch-Jozsa algorithm offers exponential speed-up by determining the output through the measurement of n( = 2)-bits

### **Deutsch Algorithm reference -> n = 1 case**

In [25]:
'''def main():
    # Choose qubits to use.
    q0, q1 = cirq.LineQubit.range(2)

    # Pick a secret 2-bit function and create a circuit to query the oracle.
    secret_function = [random.randint(0, 1) for _ in range(2)]

    oracle = make_oracle(q0, q1, secret_function)
    print(f"Secret function:\nf(x) = <{', '.join(str(e) for e in secret_function)}>")

    # Embed the oracle into a quantum circuit querying it exactly once.
    circuit = make_deutsch_circuit(q0, q1, oracle)
    print('Circuit:')
    print(circuit)

    # Simulate the circuit.
    simulator = cirq.Simulator()
    result = simulator.run(circuit)
    print('Result of f(0)⊕f(1):')
    print(result)


def make_oracle(q0, q1, secret_function):
    """Gates implementing the secret function f(x)."""

    # coverage: ignore
    if secret_function[0]:
        yield [CNOT(q0, q1), X(q1)]

    if secret_function[1]:
        yield CNOT(q0, q1)


def make_deutsch_circuit(q0, q1, oracle):
    c = cirq.Circuit()

    # Initialize qubits.
    c.append([X(q1), H(q1), H(q0)])

    # Query oracle.
    c.append(oracle)

    # Measure in X basis.
    c.append([H(q0), measure(q0, key='result')])
    return c
    
if __name__ == '__main__':
    main()'''